<a href="https://colab.research.google.com/github/zqiaohe/CupIT2020/blob/master/ClusteringData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     |████████████████████████████████| 64.4MB 47kB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from catboost import CatBoostRegressor

In [3]:

import sys
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize

import matplotlib.pyplot as plt
import datetime
%pylab inline
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

Populating the interactive namespace from numpy and matplotlib


In [0]:
def dateparse (datetime):    
    return pd.Timestamp(datetime)
data = pd.read_csv('/content/drive/My Drive/CL_megafon_Cup IT_2020_final_data (1).csv', sep = ';', parse_dates = ['time_start', 'time_end'], date_parser=dateparse)


In [0]:
coords = data.drop(['time_start', 'time_end'], axis=1)

In [0]:
coords = coords.groupby(['place_latitude', 'place_longitude']).size().reset_index(name='count')

In [0]:
cluster_coords = coords.as_matrix(columns=['place_latitude', 'place_longitude'])

In [8]:
kms_per_radian = 6371.0088
epsilon = 5 / kms_per_radian
start_time = time.time()
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(cluster_coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([cluster_coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

Number of clusters: 922


In [0]:
clustered = [0, 0, 0]
i = 1;
for cluster in clusters:
  for coord in cluster:
    coord = np.append(coord, [i])
    clustered = np.vstack((clustered, coord))
  i+=1



In [0]:
clustered = clustered.T
cols = ['place_latitude', 'place_longitude', 'cluster']
clustered = pd.DataFrame({
    'place_latitude':clustered[0],
    'place_longitude':clustered[1],
    'cluster':clustered[2]
})

In [0]:
data.index = data['time_start']
data["year"] = data.index.year
data["month"] = data.index.month
data["weekday"] = data.index.weekday
data["hour"] = data.index.hour
data["minute"] = data.index.minute
data['duration'] = data['time_end'] - data['time_start']
data['min']= data['duration'].dt.seconds/60

In [16]:
clustered_data = data.merge(clustered, on = ['place_latitude', 'place_longitude'])
clustered_data

,time_start,time_end,place_latitude,place_longitude,year,month,weekday,hour,minute,duration,min,cluster
0,2019-08-24 14:23:56,2019-08-24 18:54:47,28.102908,113.617867,2019,8,5,14,23,04:30:51,270.850000,207.0
1,2018-06-19 08:56:22,2018-06-19 12:15:26,28.102908,113.617867,2018,6,1,8,56,03:19:04,199.066667,207.0
2,2018-08-29 14:00:32,2018-08-29 14:37:51,28.102908,113.617867,2018,8,2,14,0,00:37:19,37.316667,207.0
3,2019-12-10 16:01:26,2019-12-10 16:35:22,28.102908,113.617867,2019,12,1,16,1,00:33:56,33.933333,207.0
4,2019-12-10 13:39:42,2019-12-10 17:58:22,28.999495,115.167583,2019,12,1,13,39,04:18:40,258.666667,207.0
...,...,...,...,...,...,...,...,...,...,...,...,...
78218,2018-06-25 11:25:04,2018-06-25 14:39:32,28.945221,114.357402,2018,6,0,11,25,03:14:28,194.466667,207.0
78219,2018-04-06 10:07:13,2018-04-06 16:34:40,28.619318,114.397573,2018,4,4,10,7,06:27:27,387.450000,207.0
78220,2018-07-13 22:59:34,2018-07-14 00:05:50,28.619318,114.397573,2018,7,4,22,59,01:06:16,66.266667,207.0
78221,2018-11-06 00:05:57,2018-11-06 08:50:10,28.619318,114.397573,2018,11,1,0,5,08:44:13,524.216667,207.0


In [0]:
X = clustered_data.drop(['time_end', 'min', 'duration'], axis = 1)

In [0]:
y = clustered_data['min']

In [23]:
params = {'loss_function':'MAE', 'learning_rate':1, 'depth':5}
model = CatBoostRegressor(iterations=10000,  **params, task_type='GPU')
model.fit(X, y)

Streaming output truncated to the last 5000 lines.
5001:	learn: 173.7490254	total: 22.1s	remaining: 22.1s
5002:	learn: 173.7424328	total: 22.1s	remaining: 22.1s
5003:	learn: 173.7352182	total: 22.1s	remaining: 22.1s
5004:	learn: 173.7292561	total: 22.1s	remaining: 22s
5005:	learn: 173.7215419	total: 22.1s	remaining: 22s
5006:	learn: 173.7128758	total: 22.1s	remaining: 22s
5007:	learn: 173.7003847	total: 22.1s	remaining: 22s
5008:	learn: 173.6870730	total: 22.1s	remaining: 22s
5009:	learn: 173.6724223	total: 22.1s	remaining: 22s
5010:	learn: 173.6620389	total: 22.1s	remaining: 22s
5011:	learn: 173.6515701	total: 22.1s	remaining: 22s
5012:	learn: 173.6363982	total: 22.1s	remaining: 22s
5013:	learn: 173.6269174	total: 22.1s	remaining: 22s
5014:	learn: 173.6139582	total: 22.1s	remaining: 22s
5015:	learn: 173.6108667	total: 22.1s	remaining: 22s
5016:	learn: 173.6064650	total: 22.1s	remaining: 22s
5017:	learn: 173.5894132	total: 22.2s	remaining: 22s
5018:	learn: 173.5808443	total: 22.2s	rema

In [0]:
y_true = np.array(data['min']) 
y_pred = model.predict(X)

In [0]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return 100 * (np.sum(np.abs((y_true - y_pred) / y_true))/len(y_true))

In [26]:
mean_absolute_percentage_error(y_true, y_pred)

288.7298779516185